In [1]:
%pip install plotly
import plotly.graph_objects as go
import ipywidgets as widgets
from math import sqrt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
## Loss functions

def avg(y_true, y_pred):
    return sum((t - p) for t, p in zip(y_true, y_pred)) / len(y_true)

def mse(y_true, y_pred):
    return sum((t - p)**2 for t, p in zip(y_true, y_pred)) / len(y_true)

def rmse(y_true, y_pred):
    return sqrt(sum((t - p)**2 for t, p in zip(y_true, y_pred)) / len(y_true))

def mae(y_true, y_pred):
    return sum(abs(t - p) for t, p in zip(y_true, y_pred)) / len(y_true)


In [39]:
def get_y_pred(model, x_data):
    return [model.forward(x) for x in x_data]


def create_figure(model, x_data, y_data, loss_fn_dropdown):
    # Determine initial axis limits
    x_min, x_max = min(x_data), max(x_data)
    y_min, y_max = min(y_data), max(y_data)
    y_pred = get_y_pred(model, x_data)

    # Create figure and initial trace
    fig = go.FigureWidget(
        data=[
            go.Scatter(x=x_data, y=y_data, mode="markers", name="Data", marker=dict(opacity=0.5)),
            go.Scatter(x=x_data, y=y_pred, mode="lines", name="Regression Line"),
            go.Scatter(
                x=[None], y=[None], mode="lines", line=dict(color="gray"), name="Error Lines"
            ),  # Dummy trace for error lines
        ],
        layout=go.Layout(
            title="Interactive Linear Regression",
            xaxis_title="Input",
            yaxis_title="Output",
            xaxis_range=[x_min - 2, x_max + 2],
            yaxis_range=[y_min - 2, y_max + 2],
            xaxis=dict(dtick=5),  # Set tick frequency for x-axis
            yaxis=dict(dtick=5),  # Set tick frequency for y-axis
            autosize=False,  # Disable automatic resizing
            width=16 * 50,  # Set width and height to the same value for square shape
            height=9 * 50,
            shapes=[
                dict(
                    type="line",
                    xref="x",
                    yref="y",
                    x0=x,
                    y0=y_data[i],
                    x1=x,
                    y1=y_pred[i],
                    line=dict(
                        color="gray",
                        width=1,
                    ),
                )
                for i, x in enumerate(x_data)
            ],
            annotations=[
                dict(
                    text=f"{loss_fn_dropdown.label}: {loss_fn_dropdown.value[0](y_data, y_pred):.2f}",
                    x=0.5,
                    y=0.98,
                    xref="paper",
                    yref="paper",
                    showarrow=False,
                )
            ],
        ),
    )
    return fig


def create_loss_vs_param_figure(model, loss):
    if len(vars(model)) == 1:
        pk, pv = list(vars(model).items())[0]
        return go.FigureWidget(
            data=[
                go.Scatter(x=[pv], y=[loss], mode="markers", name="Loss", marker=dict(color=["rgba(255, 0, 0, 1)"], size=[10])),
            ],
            layout=go.Layout(
                title="Loss vs Model Parameters",
                xaxis_title=pk + "*",
                yaxis_title="Loss",
                xaxis=dict(dtick=5),
                yaxis=dict(dtick=5),
                xaxis_range=[-25, 25],
                yaxis_range=[-25, 25],
                autosize=False,
                width=9 * 50,
                height=9 * 50,
            ),
        )
    elif len(vars(model)) == 2:
        pk1, pv1 = list(vars(model).items())[0]
        pk2, pv2 = list(vars(model).items())[1]
        return go.FigureWidget(
            data=[
                go.Scatter3d(x=[pv1], y=[pv2], z=[loss], mode="markers", name="Loss", marker=dict(color=["rgba(255, 0, 0, 1)"], size=[10])),
            ],
            layout=go.Layout(
                title="Loss vs Model Parameters",
                scene=dict(
                    xaxis=dict(title=pk1, dtick=5, range=[-25, 25]),
                    yaxis=dict(title=pk2, dtick=5, range=[-25, 25]),
                    zaxis=dict(title="Loss", dtick=5, range=[-25, 25]),
                ),
                autosize=False,
                width=16 * 50,
                height=9 * 50,
            ),
        )
    else:
        return go.FigureWidget(
            data=[],
            layout=go.Layout(
                title="Loss vs Model Parameters",
                autosize=False,
                width=16 * 50,
                height=9 * 50,
                annotations=[
                    dict(
                        text="More than 2 model parameters not supported",
                        showarrow=False,
                        xref="paper",
                        yref="paper",
                        x=0.5,
                        y=0.5,
                    )
                ],
            ),
        )


def create_sliders(model):
    # Define interactive sliders for each model parameter
    sliders = []
    for k, v in vars(model).items():
        if isinstance(v, float):
            sliders.append(widgets.FloatSlider(min=-20, max=20, step=0.1, value=v, description=k, continuous_update=True))
    return sliders


def create_loss_fn_dropdown():
    return widgets.Dropdown(
        options = {
            "Average Error": [avg, "rgba(0, 0, 255, 0.8)"],  # Cyan
            "Mean Squared Error": [mse, "rgba(255, 0, 255, 0.8)"],  # Magenta
            "Root Mean Squared Error": [rmse, "rgba(0, 255, 0, 0.8)"],  # Lime Green
            "Mean Absolute Error": [mae, "rgba(255, 165, 0, 0.8)"],  # Orange
        },
        value=[avg, "rgba(0, 0, 255, 0.8)"],
        description="Losses:",
    )


def plot(x_data, y_data, model):
    loss_fn_dropdown = create_loss_fn_dropdown()
    fig = create_figure(model, x_data, y_data, loss_fn_dropdown)
    loss_vs_param_fig = create_loss_vs_param_figure(model, loss_fn_dropdown.value[0](y_data, get_y_pred(model, x_data)))
    sliders = create_sliders(model)

    def update_plot(change):
        for i, slider in enumerate(sliders):
            setattr(model, slider.description, slider.value)
        y_pred = get_y_pred(model, x_data)
        loss = loss_fn_dropdown.value[0](y_data, y_pred)
        with fig.batch_update():
            fig.data[1].y = y_pred  # Update the line
            # Update error lines
            fig.layout.shapes = [
                dict(
                    type="line",
                    xref="x",
                    yref="y",
                    x0=x,
                    y0=y_data[i],
                    x1=x,
                    y1=y_pred[i],
                    line=dict(
                        color="gray",
                        width=2,
                    ),
                )
                for i, x in enumerate(x_data)
            ]
            # Update loss function annotation
            fig.layout.annotations[0].text = f"{loss_fn_dropdown.label}: {loss:.2f}"

        # Update loss vs param figure
        if len(vars(model)) == 1:
            pv = list(vars(model).values())[0]
            loss_vs_param_fig.data[0].x += (pv,)
            loss_vs_param_fig.data[0].y += (loss,)
        elif len(vars(model)) == 2:
            pv1, pv2 = list(vars(model).values())
            loss_vs_param_fig.data[0].x += (pv1,)
            loss_vs_param_fig.data[0].y += (pv2,)
            loss_vs_param_fig.data[0].z += (loss,)
        
        current_colors = list(loss_vs_param_fig.data[0].marker.color)
        current_sizes = list(loss_vs_param_fig.data[0].marker.size)
        current_colors[-1] = loss_fn_dropdown.value[1]
        if change["owner"] == loss_fn_dropdown:
            current_colors[-1] = change["old"][1]
        current_colors += ["rgba(255, 0, 0, 1)"]
        current_sizes[-1] = 6
        current_sizes += [10]
        loss_vs_param_fig.data[0].marker.color = current_colors
        loss_vs_param_fig.data[0].marker.size = current_sizes

    # Observe changes
    for slider in sliders:
        slider.observe(update_plot, names="value")
    loss_fn_dropdown.observe(update_plot, names="value")

    return widgets.HBox([widgets.VBox([loss_fn_dropdown] + [fig] + sliders), loss_vs_param_fig])

In [40]:
# Start with the constant model, first take a few data points where the x axis is the input and the y-axis is the output
# With the constant model, any new input will always give the same output
# So we are trying to find the best output we can get so as to minimize the loss function
# Let's start with a simple loss function like average of how far all the outputs are from the best output
# That is sum(yi-h)/n, but we will find that at the best h, the mean of the outputs is the minimizer and the error is always 0
# But error of 0 is not correct because that tells that our model predicts perfectly when it is not

# There will be two graphs for all the examples, one is the output vs input graph (with h* line shown)
# The second graph will be the loss function vs h* (To show how the loss function changes when we change h*)
# This tells them exactly the best h* where the loss function is minimum

# So we go to a better loss function (say) the absolute error, i.e sum(|yi-h|) / n
# Here the best h* would be the median and we can see how the loss function is piecewise

# Then we show the MSE loss function i.e. sum((yi-h)^2) / n, same process and here the min h* would be the mean


class ConstantModel:
    def __init__(self, h=2.0):
        self.h = h
    
    def forward(self, x):
        return self.h

x_data = [1, 2, 3, 4]
y_data = [6, 8, 1, 3]

model = ConstantModel()
plot(x_data, y_data, model)


In [10]:
# Simple linear regression
# So rather than predicting a constant, the model will predict an output that is a function of the input
# So in the simple linear regression model, the output function is a line with the output being mx + b
# Now instead of a single parameter h that we have to calculate, we now have to calculate the parameters m and b
# So the best way to do it is to still consider a loss function that gives you some error for your prediction and then
# find the point where the loss function is minimum. So how do we find these parameters?

class SimpleLinearRegression:
    def __init__ (self, m=0.7, c=2.2):
        self.m = m
        self.c = c
    
    def forward(self, x):
        return self.m * x + self.c

# Initial data points
x_data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
y_data = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

model = SimpleLinearRegression()
plot(x_data, y_data, model)